In [6]:
# นำเข้าไลบรารีที่จำเป็น
import pandas as pd
from pycaret.classification import *
from datetime import datetime
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# โหลดข้อมูลจากไฟล์ vl_data.csv
data = pd.read_csv('vl_data.csv')

# แปลงคอลัมน์ 'timestamp' เป็น datetime object
data['timestamp'] = pd.to_datetime(data['timestamp'])

# แปลง timestamp เป็นจำนวนวัน (ใช้ช่วงเวลาจากวันแรกในข้อมูล)
data['timestamp'] = (data['timestamp'] - data['timestamp'].min()).dt.days

# จัดเรียงข้อมูลตามเวลา
data = data.sort_values(by='timestamp')

# สร้าง Target Variable โดยใช้ pm_2_5 ของอีก 7 วันข้างหน้า
data['pm_2_5_future'] = data['pm_2_5'].shift(-7)
data['is_pm25_high'] = (data['pm_2_5_future'] > 25).astype(int)

# ลบแถวที่ไม่มีข้อมูล pm_2_5 ของสัปดาห์หน้า
data.dropna(subset=['pm_2_5_future'], inplace=True)

# เลือกเฉพาะคอลัมน์ที่ต้องการ
feature_cols = ['timestamp', 'humidity', 'temperature', 'pm_2_5', 'is_pm25_high']
data = data[feature_cols]

# เติมค่า Missing ในคอลัมน์ตัวเลขด้วยค่าเฉลี่ย
numeric_cols = data.select_dtypes(include=['number']).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# แยกข้อมูล Train/Test ก่อน Oversampling
X = data.drop('is_pm25_high', axis=1)
y = data['is_pm25_high']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Oversampling เฉพาะข้อมูล Train
smote = SMOTE(random_state=123)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# สร้างโมเดลด้วยข้อมูล Train ที่ Oversampled
clf_setup = setup(
    data=pd.concat([X_train_resampled, y_train_resampled], axis=1),
    target='is_pm25_high',
    session_id=123,
    fold=10,
    numeric_imputation='mean'
)

# สร้างโมเดล Random Forest
rf_model = create_model('rf')

# ปรับพารามิเตอร์ Regularization
tuned_rf = tune_model(
    rf_model,
    optimize='F1',
    n_iter=50,
    fold=10,
    verbose=True,
    custom_grid={
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 3],
        'class_weight': ['balanced', None]
    }
)

# ประเมินผลบน Test Set
print("ผลประเมินบน Test Set:")
evaluate_model(tuned_rf)

# ดึงผลลัพธ์การประเมินโมเดล
results = pull()

# แสดงผลลัพธ์ในรูปแบบที่ต้องการ
# ตรวจสอบจำนวนคอลัมน์ใน results
print(results.columns)

# ตั้งชื่อคอลัมน์ใหม่ตามจำนวนคอลัมน์ที่มีอยู่
if len(results.columns) == 7:
    results.columns = ['Model', 'Accuracy', 'AUC', 'Recall', 'Prec.', 'F1', 'Kappa']
elif len(results.columns) == 8:
    results.columns = ['Model', 'Accuracy', 'AUC', 'Recall', 'Prec.', 'F1', 'Kappa', 'MCC']
else:
    print("Unexpected number of columns in results.")

# แสดงผลลัพธ์
print(results)

# ทำนายผลบน Test Set
test_predictions = predict_model(tuned_rf, data=X_test)

# บันทึกโมเดล Random Forest
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
save_model(tuned_rf, f'pm25_rf_classification_model_{current_time}')

print(f"โมเดล Random Forest ถูกบันทึกเรียบร้อยแล้วในชื่อ 'pm25_rf_classification_model_{current_time}.pkl'")


,Description,Value
0,Session id,123
1,Target,is_pm25_high
2,Target type,Binary
3,Original data shape,"(1426, 5)"
4,Transformed data shape,"(1426, 5)"
5,Transformed train set shape,"(998, 5)"
6,Transformed test set shape,"(428, 5)"
7,Numeric features,4
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8400,0.9286,0.9000,0.8036,0.8491,0.6800,0.6849
1,0.8700,0.9282,0.9600,0.8136,0.8807,0.7400,0.7523
2,0.8600,0.9178,0.8800,0.8462,0.8627,0.7200,0.7206
3,0.9400,0.9694,0.9800,0.9074,0.9423,0.8800,0.8828
4,0.9000,0.9530,0.9600,0.8571,0.9057,0.8000,0.8058
5,0.8600,0.9576,0.9800,0.7903,0.8750,0.7200,0.7417
6,0.9500,0.9916,1.0000,0.9091,0.9524,0.9000,0.9045
7,0.8700,0.9306,0.9000,0.8491,0.8738,0.7400,0.7413
8,0.8687,0.9296,0.9000,0.8491,0.8738,0.7372,0.7385


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8400,0.9286,0.9000,0.8036,0.8491,0.6800,0.6849
1,0.8700,0.9282,0.9600,0.8136,0.8807,0.7400,0.7523
2,0.8600,0.9178,0.8800,0.8462,0.8627,0.7200,0.7206
3,0.9400,0.9694,0.9800,0.9074,0.9423,0.8800,0.8828
4,0.9000,0.9530,0.9600,0.8571,0.9057,0.8000,0.8058
5,0.8600,0.9576,0.9800,0.7903,0.8750,0.7200,0.7417
6,0.9500,0.9916,1.0000,0.9091,0.9524,0.9000,0.9045
7,0.8700,0.9306,0.9000,0.8491,0.8738,0.7400,0.7413
8,0.8687,0.9292,0.8800,0.8627,0.8713,0.7373,0.7374


Fitting 10 folds for each of 50 candidates, totalling 500 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
ผลประเมินบน Test Set:


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Index(['Accuracy', 'AUC', 'Recall', 'Prec.', 'F1', 'Kappa', 'MCC'], dtype='object')
       Model  Accuracy     AUC  Recall   Prec.      F1   Kappa
Fold                                                          
0     0.8400    0.9286  0.9000  0.8036  0.8491  0.6800  0.6849
1     0.8700    0.9282  0.9600  0.8136  0.8807  0.7400  0.7523
2     0.8600    0.9178  0.8800  0.8462  0.8627  0.7200  0.7206
3     0.9400    0.9694  0.9800  0.9074  0.9423  0.8800  0.8828
4     0.9000    0.9530  0.9600  0.8571  0.9057  0.8000  0.8058
5     0.8600    0.9576  0.9800  0.7903  0.8750  0.7200  0.7417
6     0.9500    0.9916  1.0000  0.9091  0.9524  0.9000  0.9045
7     0.8700    0.9306  0.9000  0.8491  0.8738  0.7400  0.7413
8     0.8687    0.9292  0.8800  0.8627  0.8713  0.7373  0.7374
9     0.9495    0.9902  0.9388  0.9583  0.9485  0.8990  0.8991
Mean  0.8908    0.9496  0.9379  0.8597  0.8961  0.7816  0.7871
Std   0.0391    0.0256  0.0424  0.0498  0.0364  0.0782  0.0765


Transformation Pipeline and Model Successfully Saved
โมเดล Random Forest ถูกบันทึกเรียบร้อยแล้วในชื่อ 'pm25_rf_classification_model_20250317_021402.pkl'


In [10]:
# นำเข้าไลบรารีที่จำเป็น
import pandas as pd
from pycaret.classification import *

# โหลดโมเดลที่บันทึกไว้
loaded_model = load_model('pm25_rf_classification_model_20250317_021402')  # เปลี่ยนชื่อไฟล์ตามที่บันทึกไว้

# โหลดข้อมูลเดิมที่ใช้ฝึกโมเดล (หรือข้อมูลที่ใช้ทดสอบ)
data = pd.read_csv('vl_data.csv')  # โหลดข้อมูลเดิมจากไฟล์

# แปลงคอลัมน์ 'timestamp' เป็น datetime object (ถ้ายังไม่แปลง)
data['timestamp'] = pd.to_datetime(data['timestamp'])

# แปลง timestamp เป็นจำนวนวัน (ตามที่ทำในการฝึกโมเดล)
data['timestamp'] = (data['timestamp'] - data['timestamp'].min()).dt.days

# เลือกคอลัมน์ที่จำเป็นสำหรับการทำนาย
feature_cols = ['timestamp', 'humidity', 'temperature', 'pm_2_5']
data = data[feature_cols]

# ทำนายผลด้วยโมเดล
predictions = predict_model(loaded_model, data=data)

# แสดงผลลัพธ์การทำนาย
print(predictions)

# ถ้าต้องการบันทึกผลลัพธ์ทำนายเป็นไฟล์ใหม่
predictions.to_csv('predictions_on_old_data.csv', index=False)


Transformation Pipeline and Model Successfully Loaded


     timestamp   humidity  temperature     pm_2_5  prediction_label  \
0            0  59.128956    30.222733  15.855718                 0   
1            1  54.113052    32.898041  16.487726                 0   
2            2  57.145958    30.867455  12.600487                 0   
3            3  52.165211    32.647743   9.318757                 0   
4            4  53.445206    32.407467  11.211834                 0   
..         ...        ...          ...        ...               ...   
966        991  42.251850    34.368073   9.977366                 0   
967        992  42.297764    34.149311   7.547813                 0   
968        993  43.659225    34.133865   9.606482                 0   
969        994  42.147102    34.648891  12.767817                 0   
970        994  43.776623    35.045628  17.502995                 0   

     prediction_score  
0                0.52  
1                0.87  
2                0.57  
3                0.95  
4                0.67  
.. 

In [8]:
import os
print(os.listdir())

['.venv', 'cleaned_vl_data.csv', 'clean_hr_data.ipynb', 'clean_vl.ipynb', 'logs.log', 'mlruns', 'pm25_et_classification_model_20250316_175922.pkl', 'pm25_lgbm_classification_model_20250316_212548.pkl', 'pm25_rf_classification_model_20250317_021402.pkl', 'train1.ipynb', 'train2.ipynb', 'train3.ipynb', 'train4.ipynb', 'train5.ipynb', 'vl_data.csv', 'vl_hr_clean.csv', 'vl_hr_clean1.csv', 'vl_hr_clean_last4months.csv', 'VL_hr_data.csv', 'website', '__pycache__']
